In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.extract_results import extract_results_to_txt_file, display_results_by_subdistribution,\
                                    display_results_by_named_entity, display_confusion_matrix

from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

from estnltk.taggers import VabamorfCorpusTagger
vm_corpus_tagger = VabamorfCorpusTagger()

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

These files don't work because the protocols are written in a different language, which the goldstandard didn't recognise, hence have no goldstandard tags.

In [3]:
files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                     'J2rva_Tyri_V22tsa_id22178_1912a.json']
'''
models = ['model_default',\
          'model_local_features_without_morph',\
          'model_morph_without_lemmas',\
          'model_morph_with_lemmas',\
          'model_morph_with_lemmas_and_sentences',\
          'model_morph_with_lemmas_and_sentences_and_gazzetteer',\
          'model_morph_with_lemmas_and_sentences_and_gazzetteer_and_global_features']
'''
models = ['model_morph_with_lemmas_and_sentences_and_gazzetteer', \
          'model_morph_with_lemmas_and_sentences_and_gazzetteer_and_global_features']

Flags:

In [4]:
use_vabamorfcorpustagger = True

In [5]:
files = {}

with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [6]:
for model_directory in models:
    for subdistribution in [1, 2, 3, 4, 5]:
        training_subdistributions = []
        for y in [1, 2, 3, 4, 5]:
            if y == subdistribution:
                subdistribution_for_testing = y
            else:
                training_subdistributions.append(y)

        # Getting the filenames to be trained on from the files dictionary.
        filenames = {key: value for key, value in files.items() if int(value) in training_subdistributions}
        
        # Creating training_texts from the aforementioned filenames.
        print("Valmistan ette treenimistekste.")
        start = time.time()
        training_texts = []
        for filename in filenames:
            with open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', filename), 'r', encoding='UTF-8') as file:
                if filename in files_not_working:
                    continue
                else:
                    tagged_text = preprocess_text(json_to_text(file.read()))
                    if use_vabamorfcorpustagger:
                        tagged_text.pop_layer('morph_analysis')
                        vm_corpus_tagger.tag([tagged_text])
                    training_texts.append(tagged_text)
        print(f"Treenimistekstid ette valmistatud {time.time() - start} sekundiga.")
        
        # Setting up the trainer and training.
        print("Alustan NerTaggeri treenimist.")
        start = time.time()
        new_model_dir = os.path.join('models', model_directory)
        modelUtil = ModelStorageUtil( new_model_dir )
        nersettings = modelUtil.load_settings()
        trainer = NerTrainer(nersettings)
        trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
        print(f"NerTagger treenitud {time.time() - start} sekundiga.")

        # Setting up the new trained nertagger and defining layers to be removed later on.
        nertagger = NerTagger(model_dir = new_model_dir)
        removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']

        # Tagging the files using the new nertagger.
        print("\n\nAlustan failide märgendamist.")
        start = time.time()
        testing_files = {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}
        for file in testing_files:
            with open(find(file.replace(".json", ".txt"), os.path.join('..', 'data', 'vallakohtufailid')), 'r', encoding='UTF-8') as f:
                text = f.read()
                if file == "Tartu_V6nnu_Ahja_id3502_1882a.txt":
                    text = text.replace('..', '. .')
                text = preprocess_text(Text(text))
                
                if use_vabamorfcorpustagger:
                    text.pop_layer('morph_analysis')
                    text = [text]
                    vm_corpus_tagger.tag( text )
                    text = text[0]
                nertagger.tag(text)
                text.add_layer(flatten(text['ner'], 'flat_ner'))
                    
                for x in removed_layers:
                    text.pop_layer(x)
                
                text_to_json(text, file=os.path.join(new_model_dir, 'vallakohtufailid-trained-nertagger', file))
                print(f'Märgendatud fail {file}')
        print(f"Failid märgendatud {time.time() - start} sekundiga.")
print("Programm on lõpetanud oma töö.")

Valmistan ette treenimistekste.
Treenimistekstid ette valmistatud 204.85402941703796 sekundiga.
Alustan NerTaggeri treenimist.
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 344037
Seconds required: 1.269

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 30999.990087
Trial #1 (eta = 0.100000): 4585.565258
Trial #2 (eta = 0.200000): 7994.301132
Trial #3 (eta = 0.400000): 16231.798084
Trial #4 (eta = 0.800000): 30915.168493
Trial #5 (eta = 1.600000): 52544.547232 (worse)
Trial #6 (eta = 0.050000): 3492.666113
Tri

***** Epoch #38 *****
Loss: 897.844473
Improvement ratio: 0.365518
Feature L2-norm: 90.706766
Learning rate (eta): 0.024953
Total number of feature updates: 617234
Seconds required for this iteration: 0.459

***** Epoch #39 *****
Loss: 880.131911
Improvement ratio: 0.347984
Feature L2-norm: 91.368963
Learning rate (eta): 0.024951
Total number of feature updates: 633477
Seconds required for this iteration: 0.456

***** Epoch #40 *****
Loss: 853.347335
Improvement ratio: 0.343619
Feature L2-norm: 92.013577
Learning rate (eta): 0.024950
Total number of feature updates: 649720
Seconds required for this iteration: 0.462

***** Epoch #41 *****
Loss: 839.280378
Improvement ratio: 0.319694
Feature L2-norm: 92.644473
Learning rate (eta): 0.024949
Total number of feature updates: 665963
Seconds required for this iteration: 0.458

***** Epoch #42 *****
Loss: 818.953268
Improvement ratio: 0.308590
Feature L2-norm: 93.258475
Learning rate (eta): 0.024948
Total number of feature updates: 682206
Seco

***** Epoch #78 *****
Loss: 453.748714
Improvement ratio: 0.138683
Feature L2-norm: 109.088227
Learning rate (eta): 0.024903
Total number of feature updates: 1266954
Seconds required for this iteration: 0.460

***** Epoch #79 *****
Loss: 450.215836
Improvement ratio: 0.130460
Feature L2-norm: 109.413260
Learning rate (eta): 0.024902
Total number of feature updates: 1283197
Seconds required for this iteration: 0.459

***** Epoch #80 *****
Loss: 443.753046
Improvement ratio: 0.127714
Feature L2-norm: 109.737523
Learning rate (eta): 0.024900
Total number of feature updates: 1299440
Seconds required for this iteration: 0.460

***** Epoch #81 *****
Loss: 438.717804
Improvement ratio: 0.136715
Feature L2-norm: 110.056093
Learning rate (eta): 0.024899
Total number of feature updates: 1315683
Seconds required for this iteration: 0.458

***** Epoch #82 *****
Loss: 439.469038
Improvement ratio: 0.105600
Feature L2-norm: 110.370333
Learning rate (eta): 0.024898
Total number of feature updates: 13

***** Epoch #120 *****
Loss: 317.821407
Improvement ratio: 0.075171
Feature L2-norm: 120.100971
Learning rate (eta): 0.024851
Total number of feature updates: 1949160
Seconds required for this iteration: 0.457

***** Epoch #121 *****
Loss: 317.451122
Improvement ratio: 0.058937
Feature L2-norm: 120.311522
Learning rate (eta): 0.024850
Total number of feature updates: 1965403
Seconds required for this iteration: 0.458

***** Epoch #122 *****
Loss: 315.593449
Improvement ratio: 0.076623
Feature L2-norm: 120.520501
Learning rate (eta): 0.024848
Total number of feature updates: 1981646
Seconds required for this iteration: 0.458

***** Epoch #123 *****
Loss: 311.891370
Improvement ratio: 0.077084
Feature L2-norm: 120.729339
Learning rate (eta): 0.024847
Total number of feature updates: 1997889
Seconds required for this iteration: 0.457

***** Epoch #124 *****
Loss: 307.146102
Improvement ratio: 0.083801
Feature L2-norm: 120.935252
Learning rate (eta): 0.024846
Total number of feature update

***** Epoch #161 *****
Loss: 259.585677
Improvement ratio: 0.012432
Feature L2-norm: 127.581462
Learning rate (eta): 0.024800
Total number of feature updates: 2615123
Seconds required for this iteration: 0.457

***** Epoch #162 *****
Loss: 252.334850
Improvement ratio: 0.057641
Feature L2-norm: 127.739262
Learning rate (eta): 0.024799
Total number of feature updates: 2631366
Seconds required for this iteration: 0.460

***** Epoch #163 *****
Loss: 251.069768
Improvement ratio: 0.052743
Feature L2-norm: 127.896668
Learning rate (eta): 0.024798
Total number of feature updates: 2647609
Seconds required for this iteration: 0.482

***** Epoch #164 *****
Loss: 253.615944
Improvement ratio: 0.040792
Feature L2-norm: 128.051969
Learning rate (eta): 0.024797
Total number of feature updates: 2663852
Seconds required for this iteration: 0.461

***** Epoch #165 *****
Loss: 250.861162
Improvement ratio: 0.045362
Feature L2-norm: 128.206799
Learning rate (eta): 0.024795
Total number of feature update

...

Märgendatud fail Viljandi_K6pu_Suure-K6pu_id4649_1883a.json
Märgendatud fail Tartu_V6nnu_Ahja_id20421_1888a.json
Märgendatud fail Harju_Kose_Palvere_id19467_1884a.json
Märgendatud fail Viljandi_Viljandi_Karula_id19345_1867a.json
Märgendatud fail P2rnu_P2rnu-Jaagupi_Soosalu_id14294_1870a.json
Märgendatud fail Harju_Rapla_Rapla_id17276_1868a.json
Märgendatud fail Tartu_Kursi_Puurmani_id3066_1873a.json
Märgendatud fail Tartu_Torma_Avinurme_id17101_1871a.json
Märgendatud fail Harju_Kose_Palvere_id22520_1886a.json
Märgendatud fail J2rva_Tyri_S2revere_id8279_1885a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17669_1885a.json
Märgendatud fail Tartu_Kodavere_Pala_id24085_1873a.json
Märgendatud fail Tartu_Torma_Avinurme_id14737_1905a.json
Märgendatud fail L22ne_Kullamaa_Piirsalu_id13504_1899a.json
Märgendatud fail L22ne_Hanila_Massu_id446_1913a.json
Märgendatud fail J2rva_Tyri_V22tsa_id18591_1895a.json
Märgendatud fail Harju_Hageri_Kohila_id21266_1885a.json
Märgendatud fail Tartu_R6ngu_Aakre_id5921

***** Epoch #31 *****
Loss: 567.900883
Improvement ratio: 0.433259
Feature L2-norm: 103.961689
Learning rate (eta): 0.049845
Total number of feature updates: 484034
Seconds required for this iteration: 0.481

***** Epoch #32 *****
Loss: 553.344835
Improvement ratio: 0.408082
Feature L2-norm: 104.726890
Learning rate (eta): 0.049841
Total number of feature updates: 499648
Seconds required for this iteration: 0.481

***** Epoch #33 *****
Loss: 534.874914
Improvement ratio: 0.405718
Feature L2-norm: 105.476197
Learning rate (eta): 0.049836
Total number of feature updates: 515262
Seconds required for this iteration: 0.482

***** Epoch #34 *****
Loss: 514.566186
Improvement ratio: 0.358929
Feature L2-norm: 106.208783
Learning rate (eta): 0.049831
Total number of feature updates: 530876
Seconds required for this iteration: 0.482

***** Epoch #35 *****
Loss: 501.793291
Improvement ratio: 0.349600
Feature L2-norm: 106.913964
Learning rate (eta): 0.049826
Total number of feature updates: 546490

***** Epoch #71 *****
Loss: 296.219635
Improvement ratio: 0.097643
Feature L2-norm: 124.189670
Learning rate (eta): 0.049648
Total number of feature updates: 1108594
Seconds required for this iteration: 0.481

***** Epoch #72 *****
Loss: 290.566352
Improvement ratio: 0.094193
Feature L2-norm: 124.534365
Learning rate (eta): 0.049643
Total number of feature updates: 1124208
Seconds required for this iteration: 0.485

***** Epoch #73 *****
Loss: 285.945363
Improvement ratio: 0.126428
Feature L2-norm: 124.869918
Learning rate (eta): 0.049638
Total number of feature updates: 1139822
Seconds required for this iteration: 0.480

***** Epoch #74 *****
Loss: 267.670008
Improvement ratio: 0.142313
Feature L2-norm: 125.204273
Learning rate (eta): 0.049633
Total number of feature updates: 1155436
Seconds required for this iteration: 0.480

***** Epoch #75 *****
Loss: 293.182367
Improvement ratio: 0.045179
Feature L2-norm: 125.533703
Learning rate (eta): 0.049628
Total number of feature updates: 11

Märgendatud fail Harju_Juuru_Kaiu_id9874_1882a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id7840_1885a.json
Märgendatud fail V6ru_R2pina_Kahkva_id8869_1888a.json
Märgendatud fail Tartu_R6ngu_Aakre_id9106_1827a.json
Märgendatud fail V6ru_Vastseliina_Misso_id8101_1885a.json
Märgendatud fail V6ru_Vastseliina_Misso_id13681_1886a.json
Märgendatud fail Tartu_N6o_Pangodi_id5262_1889a.json
Märgendatud fail Harju_Keila_Saue_id13436_1885a.json
Märgendatud fail Harju_Kose_Triigi_id9536_1868a.json
Märgendatud fail Tartu_Torma_Avinurme_id21275_1871a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id12075_1885a.json
Märgendatud fail L22ne_Martna_Martna_id23117_1873a.json
Märgendatud fail L22ne_Kullamaa_Piirsalu_id6881_1877a.json
Märgendatud fail Viru_Haljala_Vihula_id4974_1876a.json
Märgendatud fail P2rnu_Tori_Tori_id25322_1889a.json
Märgendatud fail Tartu_Kodavere_Ranna_id19673_1865a.json
Märgendatud fail J2rva_Tyri_S2revere_id9244_1885a.json
Märgendatud fail P2rnu_P2rnu-Elisabethi_Sauga_id22818_188

Märgendatud fail Harju_Hageri_Kohila_id10769_1873a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16318_1883a.json
Märgendatud fail J2rva_Anna_Eivere_id6239_1879a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id12871_1876a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16931_1886a.json
Märgendatud fail Harju_Harju-Madise_Padise-Kloostri_id2778_1867a.json
Märgendatud fail Viljandi_K6pu_Suure-K6pu_id4977_1883a.json
Märgendatud fail J2rva_Tyri_S2revere_id8673_1886a.json
Märgendatud fail V6ru_R2pina_Kahkva_id14118_1889a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id22309_1867a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id25096_1877a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id7206_1876a.json
Märgendatud fail V6ru_R2pina_Kahkva_id6348_1887a.json
Märgendatud fail Tartu_N6o_Meeri_id4980_1895a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14731_1882a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id23690_1866a.json
Märgendatud fail Harju_Kose_Palvere_id17074_1882a.json
Märgendatud fail Viljandi_Paistu_

In [7]:
for model_directory in models:
    extract_results_to_txt_file(model_directory, files)

Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
